In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:34305")
client

<Client: 'tcp://127.0.0.1:34305' processes=8 threads=40, memory=157.36 GiB>

In [2]:
client.restart()

<Client: 'tcp://127.0.0.1:36609' processes=0 threads=0, memory=0 B>

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import os.path as op
import os
import gsw
from xgcm.grid import Grid
from xmitgcm import open_mdsdataset
from fastjmd95 import rho as densjmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/tank/chaocean/qjamet/RUNS/ORAR/reruns2/'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'
savedir = '/tank/spectre/tuchida/chaocean/Snap/'

In [4]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
# Cp = 3.994e3
Cp = 3989.244953
# years = np.arange(2008,2013,dtype=int)
ySstart = -15
yNstart = 15
yend = 50
dnf = 2160
nfile = 73

In [5]:
xchunk = 100
ychunk = 100
# dCs = 1e-5
# Cs = xr.open_dataset(op.join(savedir,"Max-snap-sound-speed_1967.nc")).Cs
# Cs

In [6]:
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1419120,1576800,dnf).reshape(len(years),nfile)
dd = 0  # 
yystart = 0
ntimes[yystart,dd:]

array([1419120, 1421280, 1423440, 1425600, 1427760, 1429920, 1432080,
       1434240, 1436400, 1438560, 1440720, 1442880, 1445040, 1447200,
       1449360, 1451520, 1453680, 1455840, 1458000, 1460160, 1462320,
       1464480, 1466640, 1468800, 1470960, 1473120, 1475280, 1477440,
       1479600, 1481760, 1483920, 1486080, 1488240, 1490400, 1492560,
       1494720, 1496880, 1499040, 1501200, 1503360, 1505520, 1507680,
       1509840, 1512000, 1514160, 1516320, 1518480, 1520640, 1522800,
       1524960, 1527120, 1529280, 1531440, 1533600, 1535760, 1537920,
       1540080, 1542240, 1544400, 1546560, 1548720, 1550880, 1553040,
       1555200, 1557360, 1559520, 1561680, 1563840, 1566000, 1568160,
       1570320, 1572480, 1574640])

In [12]:
dsg = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                              % (100)), 
                      grid_dir=gdir,
                      prefix=['snap_diagOcn'], delta_t=2e2,
                      iters=ntimes[0,0], 
                     ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,
                                           'XC':xchunk,'XG':xchunk})
grid = Grid(dsg, periodic=['X'])
Vol = (dsg.rA * dsg.drF * dsg.hFacC).where(dsg.maskC!=0.).reset_coords(drop=True)
dsg

<xarray.Dataset>
Dimensions:  (XC: 1000, YC: 900, XG: 1000, YG: 900, Z: 46, Zp1: 47, Zu: 46,
              Zl: 46)
Coordinates: (12/33)
    iter     int64 1419120
    time     timedelta64[ns] 3285 days
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * XG       (XG) >f4 262.0 262.0 262.1 262.2 262.3 ... 347.7 347.8 347.8 347.9
  * YG       (YG) >f4 -19.96 -19.87 -19.79 -19.71 ... 54.71 54.79 54.88 54.96
    ...       ...
    maskW    (Z, YC, XG) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskS    (Z, YG, XC) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    rhoRef   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    maskInW  (YC, XG) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    maskInS  (YG, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
Data variables:
    THETA    (Z, YC, XC) >f4 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    SALT     (Z, YC, XC) >f4 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    UVEL     (Z, YC, XG) >f4 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    VVEL     (Z, YG, XC) >f4 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    WVEL     (Zl, YC, XC) >f4 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Created by calling `open_mdsdataset(grid_dir='/tank/chaocea...

# Time mean

In [16]:
nmemb = 100
for itime in ntimes[yystart,:]:
    
    if itime == ntimes[yystart,0]:
        ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                     % (nmemb)), 
                             grid_dir=gdir,
                             prefix=['snap_diagOcn'], delta_t=2e2,
                             iters=itime, 
                            ).chunk({'YC':ychunk,'YG':ychunk,
                                     'XC':xchunk,'XG':xchunk})
    else:
        ds = xr.concat([ds, open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                                    % (nmemb)), 
                                            grid_dir=gdir,
                                            prefix=['snap_diagOcn'], delta_t=2e2,
                                            iters=itime, 
                                           ).chunk({'YC':ychunk,'YG':ychunk,
                                                    'XC':xchunk,'XG':xchunk})
                           ], dim='time')

dsm = ds.reset_coords(drop=True).mean('time').compute().chunk({'YC':ychunk,'XC':xchunk})

pres = grav*np.abs(-ds.Z)
theta = dsm.THETA.where(ds.maskC!=0.)
salt = dsm.SALT.where(ds.maskC!=0.)
    
rho = xr.apply_ufunc(densjmd95, salt, theta, 
                         pres, 
                         dask='parallelized', output_dtypes=[float,]
                        )
b = -grav * (rho - rhoConst) / rhoConst
    
b.to_dataset(name='BUOY').to_zarr(op.join(savedir,'%04d/Time-decomp/Mean/BUOY.zarr' 
                                              % (years[0])),
                                      mode='w')

In [17]:
B = xr.open_zarr(op.join(savedir,'%04d/Time-decomp/Mean/BUOY.zarr' 
                                              % (years[0]))
                ).BUOY
B

<xarray.DataArray 'BUOY' (Z: 46, YC: 900, XC: 1000)>
dask.array<open_dataset-1741edb1f98c1092ec0d89f4ce5a157cBUOY, shape=(46, 900, 1000), dtype=float64, chunksize=(46, 100, 100), chunktype=numpy.ndarray>
Coordinates:
    Depth    (YC, XC) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    PHrefC   (Z) float32 dask.array<chunksize=(46,), meta=np.ndarray>
  * XC       (XC) float32 262.0 262.1 262.2 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) float32 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * Z        (Z) float32 -3.047 -9.454 -16.36 ... -5.625e+03 -5.875e+03
    drF      (Z) float32 dask.array<chunksize=(46,), meta=np.ndarray>
    hFacC    (Z, YC, XC) float32 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskC    (Z, YC, XC) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    rA       (YC, XC) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    rhoRef   (Z) float32 dask.array<chunksize=(46,), meta=np.ndarray>

In [18]:
ymax = 43
ymin = 14
xmin = 270
xmax = 337

(((grid.interp(dsm.WVEL,'Z',boundary='extend')*B).where(ds.maskC!=0.)*Vol).sel(
            Z=slice(None,-1e3),YC=slice(ymin,ymax),
            XC=slice(xmin,xmax)
            ).sum(['Z','YC','XC'],skipna=True)
 / Vol.sel(
           Z=slice(None,-1e3),YC=slice(ymin,ymax),
           XC=slice(xmin,xmax)
          ).sum(['Z','YC','XC'],skipna=True)
).compute()

<xarray.DataArray ()>
array(5.80081375e-08)